### main

In [38]:
import os
import numpy as np
import pandas as pd
from osgeo import gdal
from sklearn.cluster import KMeans

SEED = 42

In [33]:
IMG_DIR = "data/images/high_resolution/SENTINEL-2B_MSI_20210511_084252/"

bands_filenames = [
    "SENTINEL-2B_MSI_20210511_084252_channel2_1.tif",
    "SENTINEL-2B_MSI_20210511_084252_channel3_1.tif",
    "SENTINEL-2B_MSI_20210511_084252_channel4_1.tif",
    "SENTINEL-2B_MSI_20210511_084252_channel8_1.tif",
]

bands_names = ["Blue", "Green", "Red", "VNIR"]

In [34]:
data = pd.DataFrame(columns=bands_names)

In [35]:
%%time

for band_name, band_filename in zip(bands_names, bands_filenames):
    band_path = os.path.join(IMG_DIR, band_filename)
    band_img = gdal.Open(band_path, gdal.GA_ReadOnly)

    x_size, y_size = band_img.RasterXSize, band_img.RasterYSize

    band = band_img.GetRasterBand(1)
    band_data = band.ReadAsArray(0, 0, x_size, y_size)

    band_col = band_data.ravel()
    data[band_name] = band_col

    band_img = band = band_col = None

CPU times: total: 53.5 s
Wall time: 1min 19s


In [36]:
data

,Blue,Green,Red,VNIR
0,998,750,605,654
1,1008,746,612,682
2,1008,758,612,699
3,1016,748,610,703
4,1022,765,603,717
...,...,...,...,...
120560395,837,628,496,376
120560396,823,634,496,395
120560397,819,628,495,399
120560398,827,626,490,440


In [37]:
%%time

data.to_csv('data/SENTINEL-2B_MSI_20210511_084252_Blue_Red_Green_VNIR.csv', index=False)

CPU times: total: 3min 35s
Wall time: 3min 52s


In [39]:
%%time

kmeans = KMeans(n_clusters=20, random_state=SEED).fit(data)

CPU times: total: 4h 38min 36s
Wall time: 5h 15min 1s


In [40]:
kmeans.labels_

array([3, 3, 3, ..., 9, 9, 9])

In [41]:
%%time

data.to_csv('data/SENTINEL-2B_MSI_20210511_084252_kmeans_labels_trained_on_Blue_Red_Green_VNIR.csv', index=False)

CPU times: total: 3min 32s
Wall time: 3min 58s


In [ ]:
[
    "channel2",
    "channel3",
    "channel4",
    "channel8_",
]

In [12]:
for img in sentinel_imgs:
    img_path = os.path.join(HIGH_RES_IMG_DIR, img)
    bands_filenames = list(filter(lambda x: any(band_name in x for band_name in bands_names), os.listdir(img_path)))
    print(img, bands_filenames)

SENTINEL-2B_MSI_20210325_085404 ['SENTINEL-2B_MSI_20210325_085404_channel2_1.tif', 'SENTINEL-2B_MSI_20210325_085404_channel3_1.tif', 'SENTINEL-2B_MSI_20210325_085404_channel4_1.tif', 'SENTINEL-2B_MSI_20210325_085404_channel8_1.tif']
SENTINEL-2B_MSI_20190425_085420 ['SENTINEL-2B_MSI_20190425_085420_channel2_1.tif', 'SENTINEL-2B_MSI_20190425_085420_channel3_1.tif', 'SENTINEL-2B_MSI_20190425_085420_channel4_1.tif', 'SENTINEL-2B_MSI_20190425_085420_channel8_1.tif']
SENTINEL-2B_MSI_20210511_084252 ['SENTINEL-2B_MSI_20210511_084252_channel2_1.tif', 'SENTINEL-2B_MSI_20210511_084252_channel3_1.tif', 'SENTINEL-2B_MSI_20210511_084252_channel4_1.tif', 'SENTINEL-2B_MSI_20210511_084252_channel8_1.tif']
SENTINEL-2B_MSI_20211008_084414 ['SENTINEL-2B_MSI_20211008_084414_channel2_1.tif', 'SENTINEL-2B_MSI_20211008_084414_channel3_1.tif', 'SENTINEL-2B_MSI_20211008_084414_channel4_1.tif', 'SENTINEL-2B_MSI_20211008_084414_channel8_1.tif']
SENTINEL-2A_MSI_20201117_084357 ['SENTINEL-2A_MSI_20201117_084357_ch

In [2]:
DATA_DIR = "data/"
IMG_DIR = "data/images/"
LOW_RES_IMG_DIR = "data/images/low_resolution/"
HIGH_RES_IMG_DIR = "data/images/high_resolution/"

sentinel_imgs = [
    "SENTINEL-2B_MSI_20210325_085404",
    "SENTINEL-2B_MSI_20190425_085420",
    "SENTINEL-2B_MSI_20210511_084252",
    "SENTINEL-2B_MSI_20211008_084414",
    "SENTINEL-2A_MSI_20201117_084357",
]

bands_names = [
    "channel2",
    "channel3",
    "channel4",
    "channel8_",
]

In [ ]:
%%time

for band_filename in bands_filenames:
    band_path = os.path.join(img_path, band_filename)
    band = gdal.Open(band_path, gdal.GA_ReadOnly)
    geotransform = band.GetGeoTransform()
    name = band_filename[-9:-6]
    x_size, y_size = band.RasterXSize, band.RasterYSize
    top_left_x, top_left_y = geotransform[0], geotransform[3]
    pixel_width, pixel_height = geotransform[1], geotransform[5]
    rotation_1, rotation_2 = geotransform[2], geotransform[4]
    print(f"name: {name}, size: {x_size}x{y_size}, origin: ({top_left_x}, {top_left_y}), pixel size: {pixel_width}x{pixel_height}, rotation: ({rotation_1}, {rotation_2})")

### auxiliary

In [5]:
os.listdir(HIGH_RES_IMG_DIR)

['LANDSAT_8_OLI_TIRS_20210604_082849',
 'LANDSAT_8_OLI_TIRS_20210706_082857',
 'SENTINEL-2A_MSI_20200901_053433',
 'SENTINEL-2A_MSI_20200921_053434',
 'SENTINEL-2A_MSI_20201117_084357',
 'SENTINEL-2A_MSI_20210327_052418',
 'SENTINEL-2A_MSI_20220524_053423',
 'SENTINEL-2B_MSI_20190425_085420',
 'SENTINEL-2B_MSI_20190813_053920',
 'SENTINEL-2B_MSI_20210325_085404',
 'SENTINEL-2B_MSI_20210511_084252',
 'SENTINEL-2B_MSI_20210531_052344',
 'SENTINEL-2B_MSI_20210703_053405',
 'SENTINEL-2B_MSI_20211008_084414']

In [6]:
list(filter(lambda x: x.endswith('.img'), os.listdir(LOW_RES_IMG_DIR)))

['2010-05-01.img',
 '2010-05-02.img',
 '2010-05-03.img',
 '2010-07-01.img',
 '2010-07-02.img',
 '2010-07-03.img',
 '2010-09-01.img',
 '2010-09-02.img',
 '2010-09-03.img',
 'lccswm2010_4.img',
 'priv_mod_v10_2011layer1.img',
 'priv_mod_v10_2011layer2.img']

In [16]:
img_path = os.path.join(HIGH_RES_IMG_DIR, high_res_img_dirs[0])
img_filenames = list(filter(lambda x: x.endswith('.tif'), os.listdir(img_path)))
img_filenames

['LANDSAT_8_OLI_TIRS_20210604_082849_lc2_ang_sen_azimuth_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_ang_sen_zenith_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_ang_sol_azimuth_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_ang_sol_zenith_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b10_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b11_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b1_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b2_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b3_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b4_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b5_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b6_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b7_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b8_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b9_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_qa_l1_px_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_lc2_qa_l1_rs_1.tif',
 'LANDSAT_8_OLI_TIRS_20210604_082849_source_1.tif',


In [15]:
bands_filenames = [
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b1_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b2_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b3_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b4_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b5_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b6_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b7_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b8_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b9_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b10_1.tif",
    "LANDSAT_8_OLI_TIRS_20210604_082849_lc2_b11_1.tif",
]

In [21]:
%%time

for band_filename in bands_filenames:
    band_path = os.path.join(img_path, band_filename)
    band = gdal.Open(band_path, gdal.GA_ReadOnly)
    geotransform = band.GetGeoTransform()
    name = band_filename[-9:-6]
    x_size, y_size = band.RasterXSize, band.RasterYSize
    top_left_x, top_left_y = geotransform[0], geotransform[3]
    pixel_width, pixel_height = geotransform[1], geotransform[5]
    rotation_1, rotation_2 = geotransform[2], geotransform[4]
    print(f"name: {name}, size: {x_size}x{y_size}, origin: ({top_left_x}, {top_left_y}), pixel size: {pixel_width}x{pixel_height}, rotation: ({rotation_1}, {rotation_2})")

name: _b1, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b2, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b3, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b4, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b5, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b6, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b7, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: _b8, size: 16181x16341, origin: (320392.5, 6318607.5), pixel size: 15.0x-15.0, rotation: (0.0, 0.0)
name: _b9, size: 8091x8171, origin: (320385.0, 6318615.0), pixel size: 30.0x-30.0, rotation: (0.0, 0.0)
name: b10, size: 8091x8171, origin: (320385.0, 6318615.0), pix